<a href="https://colab.research.google.com/github/AFKKZ/Traffic-Sign-Detection-in-Road-Videos/blob/main/Traffic%20Sign%20Dectection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Video

In [ ]:
import cv2
import numpy as np
import os

In [ ]:
#Paths for input and output videos
input_path = '/content/task1.mp4'
output_path = '/content/task1_output_sample.mp4'
template_path = '/content/Traffic_signs_filter'

#Open the video capture
cap = cv2.VideoCapture(input_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

#Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

#Add Student ID to the video
text = "522K0019_522K0022_522K0034"
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
font_thickness = 2
text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]

#Position text at the top-right corner
text_x = width - text_size[0] - 10
text_y = text_size[1] + 10

#Adjusted color ranges for red and blue
lower_red1 = np.array([0, 80, 80])
upper_red1 = np.array([15, 255, 255])
lower_red2 = np.array([160, 80, 80])
upper_red2 = np.array([179, 255, 255])
lower_blue = np.array([90, 150, 100])
upper_blue = np.array([130, 255, 255])

#Shape filtering parameters
min_area = 500  #Minimum contour area to consider
circularity_threshold = 0.7  #Threshold for circularity to detect circular signs

#Load traffic sign templates dynamically from the template folder
templates = []
template_names = []

for filename in os.listdir(template_path):
  #Check if the file is an image
  if filename.endswith(('.jpg')):
    template_img = cv2.imread(os.path.join(template_path, filename), 0)
    if template_img is not None:
      templates.append(template_img)
      template_names.append(filename)

#Function for template matching
def match_template(roi):
  for i, template in enumerate(templates):
    h, w = template.shape
    roi_resized = cv2.resize(roi, (w, h))  #Resize ROI to match template size
    res = cv2.matchTemplate(roi_resized, template, cv2.TM_CCOEFF_NORMED)
    max_val = cv2.minMaxLoc(res)[1]
    if max_val > 0.7:
      print(f"Template matched: {template_names[i]} with score {max_val}")
      return True
  return False

while cap.isOpened():
  ret, frame = cap.read()
  if not ret:
    break

  #Apply bilateral filter to reduce noise while keeping edges sharp
  filtered_frame = cv2.bilateralFilter(frame, d=9, sigmaColor=75, sigmaSpace=75)

  #Convert the frame to HSV
  hsv_frame = cv2.cvtColor(filtered_frame, cv2.COLOR_BGR2HSV)

  #Apply color masks for red and blue
  red_mask1 = cv2.inRange(hsv_frame, lower_red1, upper_red1)
  red_mask2 = cv2.inRange(hsv_frame, lower_red2, upper_red2)
  red_mask = cv2.bitwise_or(red_mask1, red_mask2)
  blue_mask = cv2.inRange(hsv_frame, lower_blue, upper_blue)

  #Apply morphological operations to reduce noise and fill gaps
  kernel = np.ones((7, 7), np.uint8)
  red_mask = cv2.morphologyEx(red_mask, cv2.MORPH_CLOSE, kernel)
  blue_mask = cv2.morphologyEx(blue_mask, cv2.MORPH_CLOSE, kernel)

  #Find contours for red and blue masks
  contours_red, _ = cv2.findContours(red_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  contours_blue, _ = cv2.findContours(blue_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

  #Function to filter contours by shape
  def is_circular(contour):
    perimeter = cv2.arcLength(contour, True)
    area = cv2.contourArea(contour)
    if perimeter == 0:
      return False
    circularity = 4 * np.pi * (area / (perimeter * perimeter))
    return circularity > circularity_threshold

  #Process red contours
  for contour in contours_red:
    if cv2.contourArea(contour) > min_area and is_circular(contour):
      x, y, w, h = cv2.boundingRect(contour)
      roi = frame[y:y+h, x:x+w]
      roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
      if match_template(roi_gray):
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

  #Process blue contours
  for contour in contours_blue:
    if cv2.contourArea(contour) > min_area and is_circular(contour):
      x, y, w, h = cv2.boundingRect(contour)
      roi = frame[y:y+h, x:x+w]
      roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
      if match_template(roi_gray):
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

  cv2.putText(frame, text, (text_x, text_y), font, font_scale, (255, 0, 0), font_thickness)
  #Write the processed frame to the output video
  out.write(frame)

#Release the video objects
cap.release()
out.release()

print("Processing complete. Output saved to", output_path)

Template matched: traffic_sign_39.jpg with score 0.8697137236595154
Template matched: traffic_sign_39.jpg with score 0.8983470797538757
Template matched: traffic_sign_39.jpg with score 0.8596417307853699
Template matched: traffic_sign_39.jpg with score 0.8645234107971191
Template matched: traffic_sign_39.jpg with score 0.8805254101753235
Template matched: traffic_sign_39.jpg with score 0.8720105290412903
Template matched: traffic_sign_39.jpg with score 0.8984013795852661
Template matched: traffic_sign_39.jpg with score 0.8974938988685608
Template matched: traffic_sign_39.jpg with score 0.8744507431983948
Template matched: traffic_sign_39.jpg with score 0.871518611907959
Template matched: traffic_sign_39.jpg with score 0.8693535923957825
Template matched: traffic_sign_39.jpg with score 0.8895957469940186
Template matched: traffic_sign_39.jpg with score 0.9049448370933533
Template matched: traffic_sign_39.jpg with score 0.8846350908279419
Template matched: traffic_sign_39.jpg with score 